In [ ]:
## https://docs.scipy.org/doc/scipy/tutorial/optimize.html

## Present the formulation

$$
f_{105}(\mathbf{x}) \;=\; \sum_{i=1}^{D-1} \left[ 100\left(x_{i+1} - x_i^2\right)^2 \;+\; \left(x_i - 1\right)^2 \right]
$$

In [1]:
import numpy as np
import time
from scipy.optimize import minimize

# Combined Rosenbrock function
def rosenbrock(x):
    fval = sum(100.0 * (x[1:] - x[:-1]**2.0)**2.0 + (1 - x[:-1])**2.0)

    xm = x[1:-1]
    xm_m1 = x[:-2]
    xm_p1 = x[2:]
    grad = np.zeros_like(x)
    grad[1:-1] = 200 * (xm - xm_m1**2) - 400 * (xm_p1 - xm**2) * xm - 2 * (1 - xm)
    grad[0] = -400 * x[0] * (x[1] - x[0]**2) - 2 * (1 - x[0])
    grad[-1] = 200 * (x[-1] - x[-2]**2)

    n = len(x)
    H = np.zeros((n, n))
    for i in range(n):
        if i > 0:
            H[i, i - 1] = -400 * x[i - 1]
        if i < n - 1:
            H[i, i] = 1200 * x[i]**2 - 400 * x[i + 1] + 2
            H[i, i + 1] = -400 * x[i]
        else:
            H[i, i] = 200
    for i in range(n - 1):
        H[i + 1, i] = H[i, i + 1]

    return fval, grad, H

# Initial point
x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])

# List of methods to compare
methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS", "jac": lambda x: rosenbrock(x)[1]},
    "Newton-CG": {"method": "Newton-CG", "jac": lambda x: rosenbrock(x)[1], "hess": lambda x: rosenbrock(x)[2]},
    "trust-ncg": {"method": "trust-ncg", "jac": lambda x: rosenbrock(x)[1], "hess": lambda x: rosenbrock(x)[2]},
    "trust-krylov": {"method": "trust-krylov", "jac": lambda x: rosenbrock(x)[1], "hess": lambda x: rosenbrock(x)[2]},
    "trust-exact": {"method": "trust-exact", "jac": lambda x: rosenbrock(x)[1], "hess": lambda x: rosenbrock(x)[2]},
}

# Executa as otimizações e armazena resultados
results = {}

for name, opts in methods.items():
    start_time = time.time()
    res = minimize(lambda x: rosenbrock(x)[0], x0, **opts, options={"disp": False})
    elapsed_time = (time.time() - start_time) * 1000  # em ms
    results[name] = {
        "x0": x0,
        "x*": res.x,
        "fval": res.fun,
        "nfev": res.nfev,
        "njev": res.get("njev", None),
        "nhev": res.get("nhev", None),
        "time": elapsed_time,
        "success": res.success
    }

# Apresenta os resultados com alinhamento
print("\n==== Rosenbrock Results ====\n")
header = f"{'Method':<20} | {'f(x*)':>12} | {'nfev':>6} | {'njev':>6} | {'nhev':>6} | {'Time (ms)':>10} | {'Success':>8}"
print(header)
print("-" * len(header))

for method, data in results.items():
    print(f"{method:<20} | {data['fval']:12.4e} | {data['nfev']:6d} | "
          f"{data['njev'] if data['njev'] is not None else '  N/A':>6} | "
          f"{data['nhev'] if data['nhev'] is not None else '  N/A':>6} | "
          f"{data['time']:10.2f} | {str(data['success']):>8}")



==== Rosenbrock Results ====

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--------------------------------------------------------------------------------------
Nelder-Mead          |   6.6175e-05 |    243 |    N/A |    N/A |      41.39 |     True
BFGS                 |   4.0131e-13 |     30 |     30 |    N/A |      15.98 |     True
Newton-CG            |   1.4917e-02 |   1003 |   1003 |   1000 |     511.52 |    False
trust-ncg            |   3.3223e-04 |   1001 |    848 |    847 |     464.16 |    False
trust-krylov         |   2.7387e-09 |    569 |    569 |    561 |    1437.18 |     True
trust-exact          |   8.2289e-10 |    655 |    645 |    655 |    1308.68 |     True


## $$
\min_{x_1, x_2} \Bigl(\cos x_1 \,\sin x_2 \;-\; \frac{x_1}{x_2^2 + 1}\Bigr)
$$

In [2]:
def ex02(x):
    x1, x2 = x

    # Função
    f = np.cos(x1) * np.sin(x2) - x1 / (x2**2 + 1)

    # Gradiente
    df_dx1 = -np.sin(x1) * np.sin(x2) - 1 / (x2**2 + 1)
    df_dx2 = np.cos(x1) * np.cos(x2) + (2 * x1 * x2) / (x2**2 + 1)**2
    g = np.array([df_dx1, df_dx2])

    # Hessiana
    d2f_dx1dx1 = -np.cos(x1) * np.sin(x2)
    d2f_dx1dx2 = -np.sin(x1) * np.cos(x2) + (2 * x2) / (x2**2 + 1)**2
    d2f_dx2dx1 = d2f_dx1dx2
    d2f_dx2dx2 = -np.cos(x1) * np.sin(x2) + \
                 (2 * x1 * (x2**2 + 1)**2 - 8 * x1 * x2**2 * (x2**2 + 1)) / (x2**2 + 1)**4
    H = np.array([
        [d2f_dx1dx1, d2f_dx1dx2],
        [d2f_dx2dx1, d2f_dx2dx2]
    ])

    return f, g, H

# Initial point
x0 = np.array([0.0, 0.0])

# List of methods to compare
methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS", "jac": lambda x: ex02(x)[1]},
    "Newton-CG": {"method": "Newton-CG", "jac": lambda x: ex02(x)[1], "hess": lambda x: ex02(x)[2]},
    "trust-ncg": {"method": "trust-ncg", "jac": lambda x: ex02(x)[1], "hess": lambda x: ex02(x)[2]},
    "trust-krylov": {"method": "trust-krylov", "jac": lambda x: ex02(x)[1], "hess": lambda x: ex02(x)[2]},
    "trust-exact": {"method": "trust-exact", "jac": lambda x: ex02(x)[1], "hess": lambda x: ex02(x)[2]},
}

# Executa as otimizações e armazena resultados
results = {}

for name, opts in methods.items():
    start_time = time.time()
    res = minimize(lambda x: ex02(x)[0], x0, **opts, options={"disp": False})
    elapsed_time = (time.time() - start_time) * 1000  # em ms
    results[name] = {
        "x0": x0,
        "x*": res.x,
        "fval": res.fun,
        "nfev": res.nfev,
        "njev": res.get("njev", None),
        "nhev": res.get("nhev", None),
        "time": elapsed_time,
        "success": res.success
    }

# Apresenta os resultados com alinhamento
print("\n==== Example 02 Results ====\n")
header = f"{'Method':<20} | {'f(x*)':>12} | {'nfev':>6} | {'njev':>6} | {'nhev':>6} | {'Time (ms)':>10} | {'Success':>8}"
print(header)
print("-" * len(header))

for method, data in results.items():
    print(f"{method:<20} | {data['fval']:12.4e} | {data['nfev']:6d} | "
          f"{data['njev'] if data['njev'] is not None else '  N/A':>6} | "
          f"{data['nhev'] if data['nhev'] is not None else '  N/A':>6} | "
          f"{data['time']:10.2f} | {str(data['success']):>8}")


==== Example 02 Results ====

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--------------------------------------------------------------------------------------
Nelder-Mead          |  -1.0457e+00 |    167 |    N/A |    N/A |      21.38 |     True
BFGS                 |  -9.6673e+22 |    248 |    236 |    N/A |      79.71 |    False
Newton-CG            |   0.0000e+00 |      1 |      1 |      1 |       0.80 |     True
trust-ncg            |  -1.6960e+05 |    399 |    396 |    395 |     149.02 |    False
trust-krylov         |   0.0000e+00 |      2 |      2 |      1 |       3.34 |    False
trust-exact          |  -2.2003e+05 |    401 |    389 |    401 |     412.53 |    False


Problem Statistics: 

\# of continuous variables: 2

\# of known solutions: 3

Global solution:

Objective function: -2.02181

Continuous variables: $x_1 = 2; x_2 = 0.10578$

In [3]:
results['trust-exact']

{'x0': array([0., 0.]),
 'x*': array([2.20032125e+05, 6.37081271e-04]),
 'fval': np.float64(-220032.03547900406),
 'nfev': 401,
 'njev': 389,
 'nhev': 401,
 'time': 412.53185272216797,
 'success': False}

In [4]:
ex02(np.array([2., 0.10578]))

(np.float64(-2.0218067833370204),
 array([-1.08494061e+00, -1.31240428e-05]),
 array([[ 0.04393797, -0.69731109],
        [-0.69731109,  3.78275021]]))

Testes de derivadas por diferenças finitas

In [5]:
# Gradiente e Hessiana numéricas
from scipy.optimize import approx_fprime
def numerical_hessian(f_grad, x, h=1e-5):
    n = len(x)
    H = np.zeros((n, n))
    fx = f_grad(x)
    for i in range(n):
        x1 = x.copy()
        x1[i] += h
        f1 = f_grad(x1)
        x2 = x.copy()
        x2[i] -= h
        f2 = f_grad(x2)
        H[:, i] = (f1 - f2) / (2 * h)
    return H

# Ponto de teste
x0 = np.array([1.0, 1.0])
eps = np.sqrt(np.finfo(float).eps)

# Avaliação da função
f_val, g_analytical, H_analytical = ex02(x0)

# Gradiente numérico
g_numeric = approx_fprime(x0, lambda x: ex02(x)[0], eps)

# Hessiana numérica via gradiente
H_numeric = numerical_hessian(lambda x: ex02(x)[1], x0)

# Diferenças absolutas
grad_diff = np.abs(g_numeric - g_analytical)
hess_diff = np.abs(H_numeric - H_analytical)

grad_diff, hess_diff


(array([7.76445575e-09, 1.14897460e-08]),
 array([[2.34168240e-12, 5.61722197e-12],
        [5.61722197e-12, 5.45773426e-11]]))

$$
\min_{x, y} 
\Bigl[
1 
+ (x + y + 1)^2 \,\bigl(19 - 14x + 3x^2 - 14y + 6xy + 3y^2\bigr)
\Bigr]
\;\times\;
\Bigl[
30 
+ (2x - 3y)^2 \,\bigl(18 - 32x^2 + 12x^2 + 48y - 36xy + 27y^2\bigr)
\Bigr]
$$

In [6]:
import numpy as np

# Função e gradiente analítico
def goldstein_price_fg(x):
    x1, x2 = x
    a = x1 + x2 + 1
    b = 2 * x1 - 3 * x2

    A = 1 + a**2 * (19 - 14 * x1 + 3 * x1**2 - 14 * x2 + 6 * x1 * x2 + 3 * x2**2)
    B = 30 + b**2 * (18 - 32 * x1 + 12 * x1**2 + 48 * x2 - 36 * x1 * x2 + 27 * x2**2)

    f = A * B

    dA_dx1 = 2 * a * (19 - 14 * x1 + 3 * x1**2 - 14 * x2 + 6 * x1 * x2 + 3 * x2**2) \
             + a**2 * (-14 + 6 * x1 + 6 * x2)
    dA_dx2 = 2 * a * (19 - 14 * x1 + 3 * x1**2 - 14 * x2 + 6 * x1 * x2 + 3 * x2**2) \
             + a**2 * (-14 + 6 * x1 + 6 * x2)

    dB_dx1 = 4 * b * (18 - 32 * x1 + 12 * x1**2 + 48 * x2 - 36 * x1 * x2 + 27 * x2**2) \
             + b**2 * (-32 + 24 * x1 - 36 * x2)
    dB_dx2 = -6 * b * (18 - 32 * x1 + 12 * x1**2 + 48 * x2 - 36 * x1 * x2 + 27 * x2**2) \
             + b**2 * (48 - 36 * x1 + 54 * x2)

    df_dx1 = dA_dx1 * B + A * dB_dx1
    df_dx2 = dA_dx2 * B + A * dB_dx2
    g = np.array([df_dx1, df_dx2])

    return f, g

# Hessiana numérica com diferenças centrais
def numerical_hessian(grad_func, x, h=1e-5):
    n = len(x)
    H = np.zeros((n, n))
    for i in range(n):
        x1 = x.copy()
        x2 = x.copy()
        x1[i] += h
        x2[i] -= h
        g1 = grad_func(x1)
        g2 = grad_func(x2)
        H[:, i] = (g1 - g2) / (2 * h)
    return H

# Interface principal
def ex03(x):
    f, g = goldstein_price_fg(x)
    H = numerical_hessian(lambda x_: goldstein_price_fg(x_)[1], x)
    return f, g, H



In [7]:
x0 = np.array([0.0, -1.0])
fval, grad, hess = ex03(x0)
print("f(x) =", fval)
print("∇f(x) =", grad)
print("∇²f(x) =\n", hess)

f(x) = 3.0
∇f(x) = [0. 0.]
∇²f(x) =
 [[ 504.0000021  -215.99999929]
 [-216.00000055  864.00000477]]


In [8]:
# Initial point
x0 = np.array([0.0, 0.0])

# List of methods to compare
methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS", "jac": lambda x: ex03(x)[1]},
    "Newton-CG": {"method": "Newton-CG", "jac": lambda x: ex03(x)[1], "hess": lambda x: ex03(x)[2]},
    "trust-ncg": {"method": "trust-ncg", "jac": lambda x: ex03(x)[1], "hess": lambda x: ex03(x)[2]},
    "trust-krylov": {"method": "trust-krylov", "jac": lambda x: ex03(x)[1], "hess": lambda x: ex03(x)[2]},
    "trust-exact": {"method": "trust-exact", "jac": lambda x: ex03(x)[1], "hess": lambda x: ex03(x)[2]},
}

# Executa as otimizações e armazena resultados
results = {}

for name, opts in methods.items():
    start_time = time.time()
    res = minimize(lambda x: ex03(x)[0], x0, **opts, options={"disp": False})
    elapsed_time = (time.time() - start_time) * 1000  # em ms
    results[name] = {
        "x0": x0,
        "x*": res.x,
        "fval": res.fun,
        "nfev": res.nfev,
        "njev": res.get("njev", None),
        "nhev": res.get("nhev", None),
        "time": elapsed_time,
        "success": res.success
    }

# Apresenta os resultados com alinhamento
print("\n==== Example 03 Results ====\n")
header = f"{'Method':<20} | {'f(x*)':>12} | {'nfev':>6} | {'njev':>6} | {'nhev':>6} | {'Time (ms)':>10} | {'Success':>8}"
print(header)
print("-" * len(header))

for method, data in results.items():
    print(f"{method:<20} | {data['fval']:12.4e} | {data['nfev']:6d} | "
          f"{data['njev'] if data['njev'] is not None else '  N/A':>6} | "
          f"{data['nhev'] if data['nhev'] is not None else '  N/A':>6} | "
          f"{data['time']:10.2f} | {str(data['success']):>8}")


==== Example 03 Results ====

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--------------------------------------------------------------------------------------
Nelder-Mead          |   3.0000e+01 |    121 |    N/A |    N/A |       7.06 |     True
BFGS                 |   3.0000e+01 |     16 |     16 |    N/A |       2.30 |     True
Newton-CG            |   3.0000e+01 |     13 |     13 |      8 |       2.16 |     True
trust-ncg            |   3.0000e+01 |     11 |      9 |      8 |       1.63 |     True
trust-krylov         |   3.0000e+01 |      7 |      7 |      6 |       4.25 |     True
trust-exact          |   3.0000e+01 |      7 |      7 |      7 |       1.85 |     True


Problem Statistics: 

\# of continuous variables: 2

\# of known solutions: 4

Global solution:

Objective function: 3

Continuous variables: $x_1 = 0.0; x_2 = -1.0$

$$
f_1(x) \;=\; -20 \, e^{-0.2 \,\sqrt{\frac{1}{D}\,\sum_{i=1}^{D} x_i^2}} 
\;-\; e^{\frac{1}{D}\,\sum_{i=1}^{D} \cos\bigl(2\pi x_i\bigr)} 
\;+\; 20 
\;+\; e
$$

In [9]:

# Valor e gradiente da função Ackley 1
def ackley1_fg(x):
    D = len(x)
    sum_sq = np.sum(x**2)
    sum_cos = np.sum(np.cos(2 * np.pi * x))
    
    term1 = -20 * np.exp(-0.2 * np.sqrt(sum_sq / D))
    term2 = -np.exp(sum_cos / D)
    f = term1 + term2 + 20 + np.e

    # Gradiente analítico
    sqrt_sum_sq = np.sqrt(sum_sq / D)
    if sqrt_sum_sq == 0:
        grad1 = 0
    else:
        grad1 = (4 * x / (D * sqrt_sum_sq)) * np.exp(-0.2 * sqrt_sum_sq)

    grad2 = (2 * np.pi / D) * np.sin(2 * np.pi * x) * np.exp(sum_cos / D)

    g = grad1 + grad2
    return f, g

# Hessiana numérica usando gradiente externo
def numerical_hessian(grad_func, x, h=1e-5):
    n = len(x)
    H = np.zeros((n, n))
    for i in range(n):
        x1 = x.copy()
        x2 = x.copy()
        x1[i] += h
        x2[i] -= h
        g1 = grad_func(x1)
        g2 = grad_func(x2)
        H[:, i] = (g1 - g2) / (2 * h)
    return H

# Interface final
def ackley1(x):
    f, g = ackley1_fg(x)
    H = numerical_hessian(lambda x_: ackley1_fg(x_)[1], x)
    return f, g, H



In [10]:
x0 = np.zeros(5)
fval, grad, hess = ackley1(x0)
print("f(x) =", fval)
print("∇f(x) =", grad)
print("∇²f(x) =\n", hess)

f(x) = 4.440892098500626e-16
∇f(x) = [0. 0. 0. 0. 0.]
∇²f(x) =
 [[178906.74089307      0.              0.              0.
       0.        ]
 [     0.         178906.74089307      0.              0.
       0.        ]
 [     0.              0.         178906.74089307      0.
       0.        ]
 [     0.              0.              0.         178906.74089307
       0.        ]
 [     0.              0.              0.              0.
  178906.74089307]]


In [11]:
# Initial point
x0 = np.ones(5)

# List of methods to compare
methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS", "jac": lambda x: ackley1(x)[1]},
    "Newton-CG": {"method": "Newton-CG", "jac": lambda x: ackley1(x)[1], "hess": lambda x: ackley1(x)[2]},
    "trust-ncg": {"method": "trust-ncg", "jac": lambda x: ackley1(x)[1], "hess": lambda x: ackley1(x)[2]},
    "trust-krylov": {"method": "trust-krylov", "jac": lambda x: ackley1(x)[1], "hess": lambda x: ackley1(x)[2]},
    "trust-exact": {"method": "trust-exact", "jac": lambda x: ackley1(x)[1], "hess": lambda x: ackley1(x)[2]},
}

# Executa as otimizações e armazena resultados
results = {}

for name, opts in methods.items():
    start_time = time.time()
    res = minimize(lambda x: ackley1(x)[0], x0, **opts, options={"disp": False})
    elapsed_time = (time.time() - start_time) * 1000  # em ms
    results[name] = {
        "x0": x0,
        "x*": res.x,
        "fval": res.fun,
        "nfev": res.nfev,
        "njev": res.get("njev", None),
        "nhev": res.get("nhev", None),
        "time": elapsed_time,
        "success": res.success
    }

# Apresenta os resultados com alinhamento
print("\n==== Example 03 Results ====\n")
header = f"{'Method':<20} | {'f(x*)':>12} | {'nfev':>6} | {'njev':>6} | {'nhev':>6} | {'Time (ms)':>10} | {'Success':>8}"
print(header)
print("-" * len(header))

for method, data in results.items():
    print(f"{method:<20} | {data['fval']:12.4e} | {data['nfev']:6d} | "
          f"{data['njev'] if data['njev'] is not None else '  N/A':>6} | "
          f"{data['nhev'] if data['nhev'] is not None else '  N/A':>6} | "
          f"{data['time']:10.2f} | {str(data['success']):>8}")


==== Example 03 Results ====

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--------------------------------------------------------------------------------------
Nelder-Mead          |   3.5745e+00 |    145 |    N/A |    N/A |     126.70 |     True
BFGS                 |   3.5745e+00 |      7 |      7 |    N/A |      13.67 |     True
Newton-CG            |   3.5745e+00 |      4 |      4 |      3 |      11.77 |     True
trust-ncg            |   3.5745e+00 |      3 |      3 |      2 |       8.79 |     True
trust-krylov         |   3.5745e+00 |      3 |      3 |      2 |      13.42 |     True
trust-exact          |   3.5745e+00 |      3 |      3 |      3 |       9.86 |     True


## Exercício 1 - Função 105 - Rosenbrock

Avaliar para diferentes valores de D

$$
f_{105}(\mathbf{x}) \;=\; \sum_{i=1}^{D-1} \left[ 100\left(x_{i+1} - x_i^2\right)^2 \;+\; \left(x_i - 1\right)^2 \right]
$$

In [12]:
import numpy as np
import time
from scipy.optimize import minimize

# Função de Rosenbrock com gradiente e hessiano
def rosenbrock(x):
    fval = sum(100.0 * (x[1:] - x[:-1]**2.0)**2.0 + (1 - x[:-1])**2.0)

    xm = x[1:-1]
    xm_m1 = x[:-2]
    xm_p1 = x[2:]
    grad = np.zeros_like(x)
    grad[1:-1] = 200 * (xm - xm_m1**2) - 400 * (xm_p1 - xm**2) * xm - 2 * (1 - xm)
    grad[0] = -400 * x[0] * (x[1] - x[0]**2) - 2 * (1 - x[0])
    grad[-1] = 200 * (x[-1] - x[-2]**2)

    n = len(x)
    H = np.zeros((n, n))
    for i in range(n):
        if i > 0:
            H[i, i - 1] = -400 * x[i - 1]
        if i < n - 1:
            H[i, i] = 1200 * x[i]**2 - 400 * x[i + 1] + 2
            H[i, i + 1] = -400 * x[i]
        else:
            H[i, i] = 200
    for i in range(n - 1):
        H[i + 1, i] = H[i, i + 1]

    return fval, grad, H

# Métodos de otimização
methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS", "jac": lambda x: rosenbrock(x)[1]},
    "Newton-CG": {"method": "Newton-CG", "jac": lambda x: rosenbrock(x)[1], "hess": lambda x: rosenbrock(x)[2]},
    "trust-ncg": {"method": "trust-ncg", "jac": lambda x: rosenbrock(x)[1], "hess": lambda x: rosenbrock(x)[2]},
    "trust-krylov": {"method": "trust-krylov", "jac": lambda x: rosenbrock(x)[1], "hess": lambda x: rosenbrock(x)[2]},
    "trust-exact": {"method": "trust-exact", "jac": lambda x: rosenbrock(x)[1], "hess": lambda x: rosenbrock(x)[2]},
}

# Dicionário para armazenar os tempos de cada método
method_times = {method: [] for method in methods}

# Executa as otimizações para D = 6 a 10
for D in range(6, 11):
    print(f"\n==== Rosenbrock Results for D = {D} ====\n")
    x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])
    if len(x0) < D:
        x0 = np.concatenate([x0, np.ones(D - len(x0))])
    else:
        x0 = x0[:D]

    results = {}
    for name, opts in methods.items():
        start_time = time.time()
        res = minimize(lambda x: rosenbrock(x)[0], x0, **opts, options={"disp": False})
        elapsed_time = (time.time() - start_time) * 1000  # em ms
        method_times[name].append(elapsed_time)  # Armazena o tempo

        results[name] = {
            "x*": res.x,
            "fval": res.fun,
            "nfev": res.nfev,
            "njev": res.get("njev", None),
            "nhev": res.get("nhev", None),
            "time": elapsed_time,
            "success": res.success
        }

    # Apresenta os resultados
    header = f"{'Method':<20} | {'f(x*)':>12} | {'nfev':>6} | {'njev':>6} | {'nhev':>6} | {'Time (ms)':>10} | {'Success':>8}"
    print(header)
    print("-" * len(header))
    for method, data in results.items():
        print(f"{method:<20} | {data['fval']:12.4e} | {data['nfev']:6d} | "
              f"{data['njev'] if data['njev'] is not None else '  N/A':>6} | "
              f"{data['nhev'] if data['nhev'] is not None else '  N/A':>6} | "
              f"{data['time']:10.2f} | {str(data['success']):>8}")

# Cálculo da média dos tempos
print("\n==== Média de Tempo por Método (D = 6 a 10) ====\n")
print(f"{'Method':<20} | {'Avg Time (ms)':>15}")
print("-" * 38)
for method, times in method_times.items():
    avg_time = sum(times) / len(times)
    print(f"{method:<20} | {avg_time:15.2f}")



==== Rosenbrock Results for D = 6 ====

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--------------------------------------------------------------------------------------
Nelder-Mead          |   3.3696e-03 |    320 |    N/A |    N/A |      12.78 |     True
BFGS                 |   3.9334e-14 |     48 |     48 |    N/A |       9.50 |     True
Newton-CG            |   1.2106e-03 |   1204 |   1204 |   1200 |     692.27 |    False
trust-ncg            |   8.6124e-06 |   1201 |   1047 |   1046 |     623.13 |    False
trust-krylov         |   3.2294e-09 |    502 |    502 |    493 |    1337.53 |     True
trust-exact          |   5.1317e-09 |    658 |    647 |    658 |    1313.29 |     True

==== Rosenbrock Results for D = 7 ====

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--------------------------------------------------------------------------------------
Nelder-Mead          |   4.6394e-03 |    441 |  

## Exercício 2- Função 114 - Scahffer 3

Testar para diferentes pontos iniciais.

$$
f_{114}(\mathbf{x}) \;=\; 0.5 
\;+\; \frac{\sin^2\left(\cos\left|x_1^2 - x_2^2\right|\right) - 0.5}{1 + 0.001\left(x_1^2 + x_2^2\right)^2}
$$

In [13]:
import numpy as np
import time
from scipy.optimize import minimize

def scahffer(x):
    x1, x2 = x[0], x[1]

    # Função
    
    # Calcula u = x1² - x2² e |u|
    u = x1**2 - x2**2
    abs_u = np.abs(u)
    
    # Termos do numerador: sin²(cos(|z|)) - 0.5
    cos_abs_u = np.cos(abs_u)
    sin_cos_abs_u_sq = np.sin(cos_abs_u)**2  # sin²(cos(|z|))
    numerator = sin_cos_abs_u_sq - 0.5
    
    # Termo do denominador: 1 + 0.001*(x1² + x2²)²
    x_sq_sum = x1**2 + x2**2
    denominator = 1 + 0.001 * (x_sq_sum ** 2)
    
    # Função final
    f = 0.5 + numerator / denominator
    
    # Gradiente
    
    # Termo v = x1² - x2² e |v|
    v = x1**2 - x2**2
    abs_v = np.abs(v)
    
    # Componentes trigonométricos
    cos_abs_v = np.cos(abs_v)
    sin_cos_abs_v = np.sin(cos_abs_v)
    cos_cos_abs_v = np.cos(cos_abs_v)
    sin_v = np.sin(v)  # Nota: sin(v), não sin(|v|)
    
    # Termos auxiliares
    x_sq_sum = x1**2 + x2**2
    k = 1 + 0.001 * (x_sq_sum ** 2)
    
    # Cálculo dos termos do numerador
    term1 = sin_cos_abs_v * cos_cos_abs_v * sin_v * k
    term2 = 0.001 * x_sq_sum * (sin_cos_abs_v**2 - 0.5)
    
    # Derivadas parciais
    df_dx1 = (-4 * x1 * (term1 + term2)) / (k ** 2)
    df_dx2  = (4 * x2 * (term1 - term2)) / (k ** 2)
        
    g = np.array([df_dx1, df_dx2])

    # Hessiana
    
    # Termos comuns
    w = x1**2 - x2**2
    abs_w = np.abs(w)
    sign_w = np.sign(w)  # Atenção: np.sign(0) = 0
    
    # Componentes trigonométricos
    cos_abs_w = np.cos(abs_w)
    sin_cos_abs_w = np.sin(cos_abs_w)
    cos_cos_abs_w = np.cos(cos_abs_w)
    sin_w = np.sin(w)  # Não usa |w|
    
    # Denominador original: 1 + 0.001*(x1² + x2²)²
    x_sq_sum = x1**2 + x2**2
    denominator = 1 + 0.001 * (x_sq_sum ** 2)
    
    # ---- Cálculo da Hessiana (termos explícitos) ----
    # ∂²f/∂x1²
    term1 = (8 * x1**2 * np.sin(abs_w)**2 * cos_cos_abs_w**2) / denominator
    term2 = (-0.004 * x_sq_sum * (sin_cos_abs_w**2 - 0.5)) / denominator**2
    term3 = (-0.008 * x1**2 * (sin_cos_abs_w**2 - 0.5)) / denominator**2
    term4 = (-8 * x1**2 * np.sin(abs_w)**2 * sin_cos_abs_w**2) / denominator
    term5 = (0.000032 * x1**2 * x_sq_sum**2 * (sin_cos_abs_w**2 - 0.5)) / denominator**3
    term6 = (-8 * x1**2 * cos_abs_w * cos_cos_abs_w * sin_cos_abs_w) / denominator
    term7 = (-4 * np.sin(abs_w) * cos_cos_abs_w * sin_cos_abs_w * sign_w) / denominator
    term8 = (0.032 * x1**2 * np.sin(abs_w) * cos_cos_abs_w * sin_cos_abs_w * sign_w * x_sq_sum) / denominator**2
    
    d2f_dx1dx1 = term1 + term2 + term3 + term4 + term5 + term6 + term7 + term8
    
    # ∂²f/∂x1∂x2 (simétrica a ∂²f/∂x2∂x1)
    term_a = (0.008 * x1 * x2 * (sin_cos_abs_w**2 - 0.5)) / denominator**2
    term_b = (0.008 * x1 * x2 * (cos_cos_abs_w**2 - 1) * (cos_abs_w**2 - 1)) / denominator
    term_c = (-0.032 * x1 * x2 * x_sq_sum**2 * (sin_cos_abs_w**2 - 0.5)) / denominator**3
    term_d = (0.008 * x1 * x2 * cos_cos_abs_w**2 * (cos_abs_w**2 - 1)) / denominator
    term_e = (0.008 * x1 * x2 * cos_abs_w * cos_cos_abs_w * sin_cos_abs_w) / denominator
    
    d2f_dx1dx2 = term_a + term_b + term_c + term_d + term_e
    
    # ∂²f/∂x2² (simétrica a ∂²f/∂x1² com ajustes de sinal)
    term_1 = (8 * x2**2 * np.sin(abs_w)**2 * cos_cos_abs_w**2) / denominator
    term_2 = (-0.004 * x_sq_sum * (sin_cos_abs_w**2 - 0.5)) / denominator**2
    term_3 = (-0.008 * x2**2 * (sin_cos_abs_w**2 - 0.5)) / denominator**2
    term_4 = (-8 * x2**2 * np.sin(abs_w)**2 * sin_cos_abs_w**2) / denominator
    term_5 = (0.000032 * x2**2 * x_sq_sum**2 * (sin_cos_abs_w**2 - 0.5)) / denominator**3
    term_6 = (8 * x2**2 * cos_abs_w * cos_cos_abs_w * sin_cos_abs_w) / denominator  # Sinal invertido
    term_7 = (4 * np.sin(abs_w) * cos_cos_abs_w * sin_cos_abs_w * sign_w) / denominator  # Sinal invertido
    term_8 = (-0.032 * x2**2 * np.sin(abs_w) * cos_cos_abs_w * sin_cos_abs_w * sign_w * x_sq_sum) / denominator**2
    
    d2f_dx2dx2 = term_1 + term_2 + term_3 + term_4 + term_5 + term_6 + term_7 + term_8
    
    # Montagem da matriz Hessiana
    H = np.array([
        [d2f_dx1dx1, d2f_dx1dx2],
        [d2f_dx1dx2, d2f_dx2dx2]
    ])

    return f, g, H

# Lista de pontos iniciais para testar
initial_points = [
    np.array([2.5, 1.75]),
    np.array([0.0, 0.0]),
    np.array([3.0, -4.0]),
    np.array([-1.5, 2.5]),
    np.array([5.0, 5.0])
]

# Métodos de otimização
methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS", "jac": lambda x: scahffer(x)[1]},
    "Newton-CG": {"method": "Newton-CG", "jac": lambda x: scahffer(x)[1], "hess": lambda x: scahffer(x)[2]},
    "trust-ncg": {"method": "trust-ncg", "jac": lambda x: scahffer(x)[1], "hess": lambda x: scahffer(x)[2]},
    "trust-krylov": {"method": "trust-krylov", "jac": lambda x: scahffer(x)[1], "hess": lambda x: scahffer(x)[2]},
    "trust-exact": {"method": "trust-exact", "jac": lambda x: scahffer(x)[1], "hess": lambda x: scahffer(x)[2]},
}

# Dicionário para armazenar os tempos de cada método
method_times = {method: [] for method in methods}

# Loop sobre os pontos iniciais
for x0 in initial_points:
    print(f"\n==== Exercício 2- Função 114 - Scahffer 3 (x0 = {x0}) ====\n")
    
    results = {}
    for name, opts in methods.items():
        start_time = time.time()
        res = minimize(lambda x: scahffer(x)[0], x0, **opts, options={"disp": False})
        elapsed_time = (time.time() - start_time) * 1000  # em ms
        
        method_times[name].append(elapsed_time)  # Armazena o tempo
        
        results[name] = {
            "x*": res.x,
            "fval": res.fun,
            "nfev": res.nfev,
            "njev": res.get("njev", None),
            "nhev": res.get("nhev", None),
            "time": elapsed_time,
            "success": res.success
        }

    # Exibe os resultados para o ponto inicial atual
    header = f"{'Method':<20} | {'f(x*)':>12} | {'nfev':>6} | {'njev':>6} | {'nhev':>6} | {'Time (ms)':>10} | {'Success':>8}"
    print(header)
    print("-" * len(header))
    for method, data in results.items():
        print(f"{method:<20} | {data['fval']:12.4e} | {data['nfev']:6d} | "
              f"{data['njev'] if data['njev'] is not None else '  N/A':>6} | "
              f"{data['nhev'] if data['nhev'] is not None else '  N/A':>6} | "
              f"{data['time']:10.2f} | {str(data['success']):>8}")
    print("\n" + "-" * 80 + "\n")

# Cálculo da média dos tempos
print("\n==== Média de Tempo por Método ====\n")
print(f"{'Method':<20} | {'Avg Time (ms)':>15}")
print("-" * 38)
for method, times in method_times.items():
    avg_time = sum(times) / len(times)
    print(f"{method:<20} | {avg_time:15.2f}")


==== Exercício 2- Função 114 - Scahffer 3 (x0 = [2.5  1.75]) ====

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--------------------------------------------------------------------------------------
Nelder-Mead          |   1.0857e-02 |    160 |    N/A |    N/A |      27.41 |     True
BFGS                 |   1.0857e-02 |     41 |     41 |    N/A |      17.45 |     True
Newton-CG            |   2.5423e-02 |    406 |    406 |    400 |     255.67 |    False
trust-ncg            |   2.3266e-02 |    399 |    390 |    389 |     216.03 |    False
trust-krylov         |   3.1861e-02 |    400 |    400 |    376 |     973.74 |    False
trust-exact          |   3.2986e-02 |    400 |    362 |    400 |     315.30 |    False

--------------------------------------------------------------------------------


==== Exercício 2- Função 114 - Scahffer 3 (x0 = [0. 0.]) ====

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--

## Exercício 3 - Função 142 - Streched V Sine Wave Function

Avaliar para diferentes valores de D

$$
f_{142}(\mathbf{x}) \;=\; \sum_{i=1}^{D-1} \left( x_{i+1}^2 + x_i^2 \right)^{0.25} 
\left[ \sin^2 \left\{ 50\left( x_{i+1}^2 + x_i^2 \right)^{0.1} \right\} \;+\; 0.1 \right]
$$

In [16]:
import numpy as np
import time
from scipy.optimize import minimize

# Função streched com gradiente e hessiano
def streched(x):
    D = len(x)
    fval = 0.0
    for i in range(D-1):
        xi = x[i]
        xi1 = x[i+1]
        sum_sq = xi**2 + xi1**2
        term1 = sum_sq ** 0.25
        u = 50 * (sum_sq ** 0.1)
        term2 = (np.sin(u))**2 + 0.1
        fval += term1 * term2

    grad = np.zeros_like(x)
    for i in range(D-1):
        xi = x[i]
        xi1 = x[i+1]
        sum_sq = xi**2 + xi1**2
        sum_sq_0_1 = sum_sq ** 0.1
        u = 50 * sum_sq_0_1
        sin_u = np.sin(u)
        cos_u = np.cos(u)
        term_sin_sq = sin_u**2 + 0.1
        sum_sq_0_25 = sum_sq ** 0.25
        sum_sq_minus_0_75 = sum_sq ** (-0.75)
        sum_sq_minus_0_9 = sum_sq ** (-0.9)

        # Gradiente para x[i]
        df_dxi_term1 = 0.5 * xi * sum_sq_minus_0_75 * term_sin_sq
        df_dxi_term2 = sum_sq_0_25 * 20 * xi * sum_sq_minus_0_9 * sin_u * cos_u
        grad[i] += df_dxi_term1 + df_dxi_term2

        # Gradiente para x[i+1]
        df_dxi1_term1 = 0.5 * xi1 * sum_sq_minus_0_75 * term_sin_sq
        df_dxi1_term2 = sum_sq_0_25 * 20 * xi1 * sum_sq_minus_0_9 * sin_u * cos_u
        grad[i+1] += df_dxi1_term1 + df_dxi1_term2

    # Hessiano
    n = len(x)
    H = np.zeros((n, n))
    for i in range(D-1):
        xi = x[i]
        xi1 = x[i+1]
        sum_sq = xi**2 + xi1**2
        sum_sq_0_1 = sum_sq ** 0.1
        u = 50 * sum_sq_0_1
        sin_u = np.sin(u)
        cos_u = np.cos(u)
        sum_sq_0_25 = sum_sq ** 0.25
        sum_sq_minus_0_75 = sum_sq ** (-0.75)
        sum_sq_minus_0_9 = sum_sq ** (-0.9)
        sum_sq_minus_1_75 = sum_sq ** (-1.75)
        sum_sq_minus_1_8 = sum_sq ** (-1.8)
        sum_sq_minus_1_9 = sum_sq ** (-1.9)

        # Termos para H[i, i]
        d2A_dxi2 = 0.5 * sum_sq_minus_0_75 - 0.75 * xi**2 * sum_sq_minus_1_75
        term1_H_ii = d2A_dxi2 * term_sin_sq
        term2_H_ii = 2 * (0.5 * xi * sum_sq_minus_0_75) * (20 * xi * sum_sq_minus_0_9 * sin_u * cos_u)
        dv_dxi = sum_sq_minus_0_9 - 1.8 * xi**2 * sum_sq_minus_1_9
        dw_dxi = (cos_u**2 - sin_u**2) * 10 * xi * sum_sq_minus_0_9
        d2B_dxi2 = 20 * (dv_dxi * sin_u * cos_u + xi * sum_sq_minus_0_9 * dw_dxi)
        term3_H_ii = sum_sq_0_25 * d2B_dxi2
        H[i, i] += term1_H_ii + term2_H_ii + term3_H_ii

        # Termos para H[i+1, i+1]
        d2A_dxi1_2 = 0.5 * sum_sq_minus_0_75 - 0.75 * xi1**2 * sum_sq_minus_1_75
        term1_H_i1i1 = d2A_dxi1_2 * term_sin_sq
        term2_H_i1i1 = 2 * (0.5 * xi1 * sum_sq_minus_0_75) * (20 * xi1 * sum_sq_minus_0_9 * sin_u * cos_u)
        dv_dxi1 = sum_sq_minus_0_9 - 1.8 * xi1**2 * sum_sq_minus_1_9
        dw_dxi1 = (cos_u**2 - sin_u**2) * 10 * xi1 * sum_sq_minus_0_9
        d2B_dxi1_2 = 20 * (dv_dxi1 * sin_u * cos_u + xi1 * sum_sq_minus_0_9 * dw_dxi1)
        term3_H_i1i1 = sum_sq_0_25 * d2B_dxi1_2
        H[i+1, i+1] += term1_H_i1i1 + term2_H_i1i1 + term3_H_i1i1

        # Termos para H[i, i+1] e H[i+1, i]
        d2A_dxixi1 = -0.75 * xi * xi1 * sum_sq_minus_1_75
        term1_H_ij = d2A_dxixi1 * term_sin_sq
        term2_H_ij = (0.5 * xi * sum_sq_minus_0_75) * (20 * xi1 * sum_sq_minus_0_9 * sin_u * cos_u) + (0.5 * xi1 * sum_sq_minus_0_75) * (20 * xi * sum_sq_minus_0_9 * sin_u * cos_u)
        dv_dxi_xi1 = -1.8 * xi * xi1 * sum_sq_minus_1_9
        dw_dxi_xi1 = (cos_u**2 - sin_u**2) * 10 * xi1 * sum_sq_minus_0_9
        d2B_dxixi1 = 20 * (dv_dxi_xi1 * sin_u * cos_u + xi * sum_sq_minus_0_9 * dw_dxi_xi1)
        term3_H_ij = sum_sq_0_25 * d2B_dxixi1
        H_ij = term1_H_ij + term2_H_ij + term3_H_ij
        H[i, i+1] += H_ij
        H[i+1, i] += H_ij

    return fval, grad, H

# Métodos de otimização
methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS", "jac": lambda x: streched(x)[1]},
    "Newton-CG": {"method": "Newton-CG", "jac": lambda x: streched(x)[1], "hess": lambda x: streched(x)[2]},
    "trust-ncg": {"method": "trust-ncg", "jac": lambda x: streched(x)[1], "hess": lambda x: streched(x)[2]},
    "trust-krylov": {"method": "trust-krylov", "jac": lambda x: streched(x)[1], "hess": lambda x: streched(x)[2]},
    "trust-exact": {"method": "trust-exact", "jac": lambda x: streched(x)[1], "hess": lambda x: streched(x)[2]},
}

# Dicionário para armazenar os tempos de cada método
method_times = {method: [] for method in methods}

# Executa as otimizações para D = 6 a 10
for D in range(6, 11):
    print(f"\n==== Streched Results for D = {D} ====\n")
    x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])
    if len(x0) < D:
        x0 = np.concatenate([x0, np.ones(D - len(x0))])
    else:
        x0 = x0[:D]

    results = {}
    for name, opts in methods.items():
        start_time = time.time()
        res = minimize(lambda x: streched(x)[0], x0, **opts, options={"disp": False})
        elapsed_time = (time.time() - start_time) * 1000  # em ms
        method_times[name].append(elapsed_time)

        results[name] = {
            "x*": res.x,
            "fval": res.fun,
            "nfev": res.nfev,
            "njev": res.get("njev", None),
            "nhev": res.get("nhev", None),
            "time": elapsed_time,
            "success": res.success
        }

    # Apresenta os resultados
    header = f"{'Method':<20} | {'f(x*)':>12} | {'nfev':>6} | {'njev':>6} | {'nhev':>6} | {'Time (ms)':>10} | {'Success':>8}"
    print(header)
    print("-" * len(header))
    for method, data in results.items():
        print(f"{method:<20} | {data['fval']:12.4e} | {data['nfev']:6d} | "
              f"{data['njev'] if data['njev'] is not None else '  N/A':>6} | "
              f"{data['nhev'] if data['nhev'] is not None else '  N/A':>6} | "
              f"{data['time']:10.2f} | {str(data['success']):>8}")

# Média dos tempos
print("\n==== Média de Tempo por Método (D = 6 a 10) ====\n")
print(f"{'Method':<20} | {'Avg Time (ms)':>15}")
print("-" * 38)
for method, times in method_times.items():
    avg_time = sum(times) / len(times)
    print(f"{method:<20} | {avg_time:15.2f}")


==== Streched Results for D = 6 ====

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--------------------------------------------------------------------------------------
Nelder-Mead          |   6.6668e-01 |    339 |    N/A |    N/A |      40.74 |     True
BFGS                 |   5.6334e-01 |     19 |     19 |    N/A |      14.75 |     True
Newton-CG            |   6.4856e-01 |     17 |     17 |     16 |      17.97 |     True
trust-ncg            |   6.4856e-01 |     19 |     18 |     17 |      18.06 |     True
trust-krylov         |   6.0919e-01 |     12 |     12 |     10 |      32.04 |     True
trust-exact          |   6.0919e-01 |     12 |      8 |     12 |      31.49 |     True

==== Streched Results for D = 7 ====

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--------------------------------------------------------------------------------------
Nelder-Mead          |   7.6647e-01 |    351 |    N/

## Exercício 4 - Função 61 - Hansen

A partir de diferentes pontos iniciais, encontrar ao menos dois dos mínimos globais.

$$
f_{61}(x) \;=\; \left( \sum_{i=0}^{4} (i+1)\cos\left(ix_1 + i + 1\right) \right) \left( \sum_{j=0}^{4} (j+1)\cos\left((j+2)x_2 + j + 1\right) \right)
$$

In [17]:
import numpy as np
import time
from scipy.optimize import minimize

def hansen(x):
    x1, x2 = x[0], x[1]
    
    # Primeira soma: Σ (i+1) * cos(i*x1 + i + 1) para i de 0 a 4
    sum1 = sum((i + 1) * np.cos(i * x1 + i + 1) for i in range(5))
    
    # Segunda soma: Σ (j+1) * cos((j+2)*x2 + j + 1) para j de 0 a 4
    sum2 = sum((j + 1) * np.cos((j + 2) * x2 + j + 1) for j in range(5))
    
    # Função final
    f = sum1 * sum2
    
    # Gradiente
    grad_x1 = sum(-(i + 1) * i * np.sin(i * x1 + i + 1) for i in range(5)) * sum2
    grad_x2 = sum1 * sum(-(j + 1) * (j + 2) * np.sin((j + 2) * x2 + j + 1) for j in range(5))
    g = np.array([grad_x1, grad_x2])
    
    # Hessiana (simplificada)
    H = np.zeros((2, 2))
    H[0, 0] = sum(-(i + 1) * i**2 * np.cos(i * x1 + i + 1) for i in range(5)) * sum2  # ∂²f/∂x1²
    H[1, 1] = sum1 * sum(-(j + 1) * (j + 2)**2 * np.cos((j + 2) * x2 + j + 1) for j in range(5))  # ∂²f/∂x2²
    H[0, 1] = H[1, 0] = grad_x1 * grad_x2 / (sum1 * sum2) if sum1 * sum2 != 0 else 0  # Aproximação
    
    return f, g, H

# Lista de pontos iniciais para testar
initial_points = [
    np.array([0.0, 0.0]),
    np.array([-2.0, 1.0]),
    np.array([3.0, -3.0]),
    np.array([1.5, 2.0]),
    np.array([-1.0, -2.0])
]

# Métodos de otimização
methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS", "jac": lambda x: hansen(x)[1]},
    "Newton-CG": {"method": "Newton-CG", "jac": lambda x: hansen(x)[1], "hess": lambda x: hansen(x)[2]},
    "trust-ncg": {"method": "trust-ncg", "jac": lambda x: hansen(x)[1], "hess": lambda x: hansen(x)[2]},
    "trust-krylov": {"method": "trust-krylov", "jac": lambda x: hansen(x)[1], "hess": lambda x: hansen(x)[2]},
    "trust-exact": {"method": "trust-exact", "jac": lambda x: hansen(x)[1], "hess": lambda x: hansen(x)[2]},
}

method_times = {method: [] for method in methods}

# Execução para cada ponto inicial
for x0 in initial_points:
    print(f"\n==== Função Hansen - Mínimos Globais (x0 = {x0}) ====\n")
    
    results = {}
    for name, opts in methods.items():
        start_time = time.time()
        res = minimize(lambda x: hansen(x)[0], x0, **opts, options={"disp": False})
        elapsed_time = (time.time() - start_time) * 1000
        
        method_times[name].append(elapsed_time)
        
        results[name] = {
            "x*": res.x,
            "fval": res.fun,
            "nfev": res.nfev,
            "njev": res.get("njev", None),
            "nhev": res.get("nhev", None),
            "time": elapsed_time,
            "success": res.success
        }

    # Exibe resultados
    header = f"{'Method':<20} | {'f(x*)':>12} | {'nfev':>6} | {'njev':>6} | {'nhev':>6} | {'Time (ms)':>10} | {'Success':>8}"
    print(header)
    print("-" * len(header))
    for method, data in results.items():
        print(f"{method:<20} | {data['fval']:12.4e} | {data['nfev']:6d} | "
              f"{data['njev'] if data['njev'] is not None else '  N/A':>6} | "
              f"{data['nhev'] if data['nhev'] is not None else '  N/A':>6} | "
              f"{data['time']:10.2f} | {str(data['success']):>8}")
    print("\n" + "-" * 80 + "\n")

# Média dos tempos
print("\n==== Média de Tempo por Método ====\n")
print(f"{'Method':<20} | {'Avg Time (ms)':>15}")
print("-" * 38)
for method, times in method_times.items():
    avg_time = sum(times) / len(times)
    print(f"{method:<20} | {avg_time:15.2f}")


==== Função Hansen - Mínimos Globais (x0 = [0. 0.]) ====

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--------------------------------------------------------------------------------------
Nelder-Mead          |  -3.8579e+01 |    118 |    N/A |    N/A |      10.37 |     True
BFGS                 |  -1.3257e+01 |     12 |     12 |    N/A |       1.81 |     True
Newton-CG            |  -3.8579e+01 |      7 |      7 |      6 |       1.18 |     True
trust-ncg            |  -3.8579e+01 |      7 |      6 |      5 |       1.79 |     True
trust-krylov         |  -3.8579e+01 |      8 |      8 |      6 |      30.53 |     True
trust-exact          |  -3.8579e+01 |      7 |      6 |      7 |       8.04 |     True

--------------------------------------------------------------------------------


==== Função Hansen - Mínimos Globais (x0 = [-2.  1.]) ====

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--------------